# Git Importations 

In [96]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [97]:
%cd drive/My Drive/INF8801A/rgbd_semantic/code

[Errno 2] No such file or directory: 'drive/My Drive/INF8801A/rgbd_semantic/code'
/content/drive/My Drive/INF8801A/rgbd_semantic/code


# Importations

In [98]:
from importations import * 
from utils import * 
from cbr import * 
from cbr_t import * 
from encoder import * 
from decoder import * 
from fusenet import *
from evaluation import * 
from gcn_encoder import * 
from gcn_decoder import * 
from gcn_module import * 
%load_ext tensorboard

# Device 
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Load the dataset

In [99]:
train = load_obj('train')
val = load_obj('validation')
test = load_obj('test')

In [100]:
''' Dataset  '''
X_train , y_train = get_train_val_test(train, is_torch = True  ) 
X_val, y_val = get_train_val_test(val, is_torch = True )
X_test, y_test = get_train_val_test(test, is_torch = True)

''' One hot labels '''
# Training set
# y_train_hot = load_obj('train_hot_labels')
# Validation set 
# y_val_hot = load_obj('val_hot_labels')
# Testing set 
# y_test_hot = load_obj('test_hot_labels')




' One hot labels '

In [101]:
del train, val 

In [102]:
# Global Variables 

IMG_SIZE = np.array(X_train[0][0,:].shape)
DEPTH_SIZE = np.array(X_train[1][0,:].shape)
LABEL_SIZE = np.array(y_train[0,:].shape)


In [103]:
print(IMG_SIZE)
print(DEPTH_SIZE)
print(LABEL_SIZE)

[  3 240 320]
[  1 240 320]
[240 320]


# GCN


![Image of Yaktocat](https://raw.githubusercontent.com/clementbernardd/rgbd_semantic/main/images/gcn.png)



In [132]:
class GCN(nn.Module) : 
  ''' GCN network implementation '''
  def __init__(self,N , name = 'gcn') : 
    ''' 
      Inputs : 
        - N : The number of classes in the dataset
        - name : The name of the model for the saving processing
    '''
    super(GCN , self).__init__()
    self.N = N
    self.name = name
    self.checkpoint_file = os.path.join('../model/' , name )

    self.gcn_encoder = GCN_ENCODER(3,1)
    self.gcn_decoder = GCN_DECODER(512, N)

  def forward(self, x):
    # Encode the data
    x, unpool, pool, gcns = self.gcn_encoder(x)
    # Add the unpool indexes
    self.gcn_decoder.unpool_indexes = unpool
    self.gcn_decoder.pool = pool
    self.gcn_decoder.gcns = gcns
    # Decode the data
    x = self.gcn_decoder(x)
    # Argmax for the prediction
    x = torch.nn.functional.log_softmax(x, dim = 1 )

    return x.type(torch.float64)

  def save_checkpoint(self) :
      print('--- Save model checkpoint ---')
      torch.save(self.state_dict(), self.checkpoint_file)

  def load_checkpoint(self) :
      print('--- Loading model checkpoint ---')
      if torch.cuda.is_available() :

          self.load_state_dict(torch.load(self.checkpoint_file))

      else :
          self.load_state_dict(torch.load(self.checkpoint_file,map_location=torch.device('cpu')))


  def load_vgg(self,dict_conversion_vgg_name , is_trainable = False ) :
    ''' Load the parameters of VGG 16 network '''
    # Load the VGG-16 Net network
    vgg16 = models.vgg16_bn( pretrained=True)
    # Get the parameters
    vgg_16_parameters = dict(vgg16.features.state_dict())
    # Get the model parameters
    params = dict(self.state_dict())
    # Loop over the dictionnary that maps the VGG16-net and our model
    for key in dict_conversion_vgg_name :
      # Set our parameters to be the one of the VGG16-net
      params[key] = vgg_16_parameters[dict_conversion_vgg_name[key]]
    # Update the paramters
    self.load_state_dict(params)
    # If not_trainable
    if not is_trainable :
      for param in self.named_parameters():
        # Make the VGG16-net parameters not trainable
        if param[0] in dict_conversion_vgg_name :
          param[1].requires_grad = False
  

In [10]:
def launch_tensorboard(name, erase = False) : 
    ''' Launch the tensorboard for the given name directory '''
    if erase : 
        %rm -r {name}
    %tensorboard --logdir {name}

In [11]:
class Train(object) : 
    ''' Object that deals the training process '''
    def __init__(self, X_train, y_train , y_train_hot,BATCH_SIZE, print_iter, criterion, \
                 optimizer, learning_rate, model,N, name, X_val = None, y_val = None  ) :
        ''' 
        Inputs : 
            - X_train : The training inputs 
            - y_train : The training labels 
            - y_train_hot : The hot-encoded training labels 
            - BATCH_SIZE : The size of the mini-batch used for the training process
            - print_iter : The number of time we plot the loss/scores during the training 
            - criterion : The criterion used 
            - optimizer : The optimizer used (like : optim.Adam)
            - learning_rate : The learning rate for the optimizer
            - model : FuseNet or GCN 
            - N : the number of classes
            - name : The name where will be stored the model parameters 
        '''
        self.X_train = X_train
        self.y_train = y_train
        self.y_train_hot = y_train_hot
        self.BATCH_SIZE = BATCH_SIZE
        self.print_iter = print_iter
        self.criterion = criterion
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.model = model(N+1, name).to(self.device)
        self.learning_rate = learning_rate
        self.optimizer = optimizer(self.model.parameters(), lr = learning_rate)
        self.name = name 
        self.N = N 
        self.X_val = X_val 
        self.y_val = y_val 

        
        self.evaluation = Evaluation()
        self.chkpt_dir = '../tmp/'+ self.name

        self.scores_train = {}
        self.scores_val = {}
        self.loss_train = {}
                
    
    def predict(self, X ) : 
        ''' Predict the output for X as inputs '''
        # Check if the size of the batch is below the BATCH_SIZE 
        if X[0].shape[0] < self.BATCH_SIZE : 
            
            predictions = convert_hot_to_squeeze_all(self.model(X).cpu())
            
        else : 
            # Split the data into mini_batch to compute the prediction 
            current_batch = 0
            # Number of images 
            n = X[0].shape[0]
            # Indexes to map the images
            idxs = np.arange(n)
            # Predictions to return
            predictions = np.zeros((n, X[0].shape[2], X[0].shape[3]))
            self.model.eval()
            # Loop over the number of mini-batches
            while current_batch < n  :
                # Get the current indexes
                index = idxs[current_batch : min(n, current_batch+self.BATCH_SIZE)] 
                # Get the current X 
                current_x = [X[0][index,:].to(self.device) , X[1][index,:].to(self.device)]
                # Without gradient 
                with torch.no_grad() :
                    # Compute the prediction
                    prediction = self.model(current_x).reshape(-1,self.N+1, 240,320)
                    # Increase the current batch
                    current_batch+= self.BATCH_SIZE
                    # Take the argmax for the prediction
                    prediction = convert_hot_to_squeeze_all(prediction.cpu())
                    # Add it to the output 
                    predictions[index, :] = prediction

            self.model.train()
        return np.array(predictions)
        
        
    def score(self, X, y) : 
        ''' Return the score for the inputs X and the output y '''
        prediction = self.predict(X)
        scores = self.evaluation.get_scores(prediction,y.cpu())
        return scores 
        
            
        
    def train(self, n_epochs, to_load = False, to_save = True, epoch_start = 0 ) : 
        ''' Train the models with the number of epochs '''
        
        if to_load : 
            self.model.load_checkpoint()
            
        n = self.X_train[0].shape[0]
        # To store the loss 
        writer = SummaryWriter(self.chkpt_dir)

        
        for e in range(epoch_start, n_epochs) : 
            current_loss = 0
            
            idxs = np.arange(n)
            # Shuffle for the mini-batch
            np.random.shuffle(idxs)
            # Initialise the counter of the mini_batch 
            current_batch = 0
            while current_batch < n :  
                # Get the mini-batches
                batch_x , batch_y = get_batch(self.X_train, self.y_train, idxs ,\
                                              current_batch, self.BATCH_SIZE)
                # Convert the labels into one hot vectors 
                batch_x = [batch_x[0].to(self.device),batch_x[1].to(self.device)]
                # batch_y = torch.from_numpy(one_hot_encode(batch_y, N= self.N+1)).to(self.device)
                batch_y = batch_y.to(self.device)
                # Increase the current batch  
                current_batch+=self.BATCH_SIZE
                # Zero the parameters of the gradients 
                self.optimizer.zero_grad()
                # Forward for the prediction
                batch_pred = self.model(batch_x)
                # Reshape the output 
                batch_pred = batch_pred.reshape(-1, self.N+1, 240,320)
                # Reshape the target 
                # batch_y = batch_y.reshape(-1, 240,320,  self.N+1).to(dtype=torch.long)
                # Compute the loss 
                loss = self.criterion(batch_pred, torch.squeeze(batch_y).long())
                # Add it to store 
                current_loss+=loss.detach().item()
                # Backward step 
                loss.backward()
                self.optimizer.step()
                
            current_loss/=self.BATCH_SIZE
            self.loss_train[e] = current_loss
            writer.add_scalar("loss/", current_loss, e )

            if e % self.print_iter == 0 : 
                scores = self.score(self.X_train , self.y_train)
                self.scores_train[e] = scores.copy()
              
                print('Epoch : {}  Loss : {:.2f}  Pixel acc : {:.2f} Mean acc : {:.2f} Mean IoU : {:.2f}'.format(e, current_loss,\
                                                                                                 scores['Pixel_accuracy'],scores['Mean_accuracy'],\
                                                                                                   scores['Mean_iou']))
                if self.X_val is not None : 
                  scores_val = self.score(self.X_val , self.y_val)
                  print('Validation set : Pixel acc : {:.2f} Mean accuracy : {:.2f} Mean IoU : {}'.format(e,\
                                                                                                 scores_val['Pixel_accuracy'],\
                                                                                                 scores_val['Mean_accuracy'],\
                                                                                                   scores_val['Mean_iou']))              

                  writer.add_scalar("scores/pixel_acc_val", scores_val['Pixel_accuracy'], e )
                  writer.add_scalar("scores/mean_acc_val", scores_val['Mean_accuracy'], e )
                  writer.add_scalar("scores/mean_iou_val", scores_val['Mean_iou'], e )

                  self.scores_val[e] = scores_val.copy()

                writer.add_scalar("scores/pixel_acc", scores['Pixel_accuracy'], e )
                writer.add_scalar("scores/mean_acc", scores['Mean_accuracy'], e )
                writer.add_scalar("scores/mean_iou", scores['Mean_iou'], e )
                                 



                self.model.save_checkpoint()
            
        writer.flush()
        writer.close()

        

## Training without VGG-16 Net transfer learning for the encoder

In [ ]:
params_train_vgg_gcn = {
    'X_train' : X_train,
    'y_train' : y_train, 
    'y_train_hot' : None, 
    'BATCH_SIZE' : 16, 
    'print_iter' : 50,
    'criterion' : nn.CrossEntropyLoss(),
    'optimizer' : optim.Adam,
    'learning_rate' : 1e-2,
    'model' : GCN,
    'N' : N,
    'name' : 'gcn',
    'X_val' : X_val,
    'y_val' : y_val
}

In [ ]:
train_gcn = Train(**params_train_vgg_gcn)

In [ ]:
train_gcn.model.load_vgg(dict_conversion_vgg_name_gcn, is_trainable=True)

In [ ]:
train_gcn.train(n_epochs = 300)